## Downloading dataset from kaggle notebooks and extracting the images and labels

In [4]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [5]:
!kaggle datasets download -d andrewmvd/road-sign-detection

 99% 216M/218M [00:11<00:00, 18.0MB/s]
100% 218M/218M [00:11<00:00, 20.6MB/s]


In [6]:
!unzip -q road-sign-detection.zip

In [8]:
!mv /content/annotations /content/labels

## Creating a function to convert VOC annotations (which are in XML) to YOLOv5 annotations (which need to be in a txt format) with its correct annotated class within the txt file. So this function will do it automatically and store it insode the labels folder

In [14]:
import os 
import xml.etree.ElementTree as ET

def convert_box(size, box):
  dw, dh = 1. / size[0], 1. / size[1]
  x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
  return x * dw, y * dh, w * dw, h * dh

def convert_voc_to_yolo():
  for anno in os.listdir('/content/labels'):
    if anno.split('.')[1] == 'xml':
      file_name = anno.split('.')[0]
      out_file = open(f'/content/labels/{file_name}.txt', 'w')

      tree = ET.parse('/content/labels/' + anno)
      root = tree.getroot()
      size = root.find('size')        
      w = int(size.find('width').text)
      h = int(size.find('height').text)

      names = ['trafficlight', 'speedlimit', 'crosswalk', 'stop']

      for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls in names and int(obj.find('difficult').text) != 1:
          xmlbox = obj.find('bndbox')
          bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
          cls_id = names.index(cls)
          out_file.write(" ".join([str(a) for a in (cls_id, *bb)]) + '\n')
  print("Conversion from VOC format to YOLO format Completed")

In [15]:
convert_voc_to_yolo()

Conversion from VOC format to YOLO format Completed


## Cloning the YOLOv5 GitHub repo to use their training scripts

In [16]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14302, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 14302 (delta 26), reused 30 (delta 15), pack-reused 14246
Receiving objects: 100% (14302/14302), 13.62 MiB | 17.28 MiB/s, done.
Resolving deltas: 100% (9836/9836), done.


In [18]:
!pip install -q -r /content/yolov5/requirements.txt

## After you run this below command, you need to change the yaml file to accomodate your new classes durig training and specify correct paths to find the annotations and images for training the model. Look at the README file to see what the VOC.yaml file should look like

In [20]:
!cp /content/yolov5/data/VOC.yaml /content/yolov5/VOC.yaml

## Running the training script by specifying the training YAML file and the weights file

In [24]:
!python /content/yolov5/train.py --img 320 --batch 16 --epochs 50 \
--data /content/yolov5/VOC.yaml --weights /content/yolov5/yolov5s.pt --workers 2

train: weights=/content/yolov5/yolov5s.pt, cfg=, data=/content/yolov5/VOC.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-18-gd7955fe Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamm

## Using the model we just trained to generate inferences

In [26]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import itertools as it
import os

## use this to detect signs in Images

In [40]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --conf 0.1 --source /content/images/road95.png

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/images/road95.png, data=yolov5/data/coco128.yaml, imgsz=[320, 320], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-18-gd7955fe Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/images/road95.png: 320x256 1 stop, 11.0ms
Speed: 0.2ms pre-process, 11.0ms inference, 1.3ms NMS per image at shape (1, 3, 320, 320)
Results saved to yolov5/runs/detect/exp7
